In [1]:
import pandas as pd
from datetime import datetime

In [2]:
defaultPath = 'D:/Gabriel/Develop/Python/IR/';
df = pd.read_csv(defaultPath + 'dados/DemonstrativoVendas01062017-10062017.csv', delimiter=';', dtype=object);
df['Parc.'] = df['Parc.'].fillna(0).astype(int);
df['Vlr. Bruto'] = df['Vlr. Bruto'].map(lambda x: x.replace('.', '').replace(',', '.')).astype(float);
df['Vlr. Comissão'] = df['Vlr. Comissão'].map(lambda x: x.replace('.', '').replace(',', '.')).astype(float);
df['Vlr. Líquido'] = df['Vlr. Líquido'].map(lambda x: x.replace('.', '').replace(',', '.')).astype(float);
df['Vlr. Rejeitado'] = df['Vlr. Rejeitado'].map(lambda x: x.replace('.', '').replace(',', '.')).astype(float);


list(df['Data'].unique())    

[nan,
 '01/06/2017',
 '02/06/2017',
 '03/06/2017',
 '04/06/2017',
 '05/06/2017',
 '06/06/2017',
 '07/06/2017',
 '08/06/2017',
 '09/06/2017',
 '10/06/2017',
 '15/06/2017',
 '16/06/2017',
 '15/08/2017']

In [4]:
df_header = df[df['ID Único EQUALS'].isnull() == True];
df_header = df_header[['Nro. Autorização', 'Vlr. Bruto', 'Vlr. Comissão', 'Vlr. Líquido', 'Vlr. Rejeitado']];
dfRows = df[df['ID Único EQUALS'].isnull() == False];
dfRows = dfRows.iloc[0:len(dfRows), 2:38];

In [5]:
list(dfRows['Data'].unique())  

['01/06/2017',
 '02/06/2017',
 '03/06/2017',
 '04/06/2017',
 '05/06/2017',
 '06/06/2017',
 '07/06/2017',
 '08/06/2017',
 '09/06/2017',
 '10/06/2017',
 '15/06/2017',
 '16/06/2017',
 '15/08/2017']

In [34]:

#list(dfRow['Data'].unique())

In [6]:
import Services.ConciliacaoRepository as repository
from Services import ConciliacaoDataMunging

conn = repository.openConn();
cursor = repository.openCursor(conn);

In [7]:
dfRows = dfRows.astype(object).where(pd.notnull(dfRows), None);
#format datetime para mm/dd/yyyy
dfRows['Data'] = dfRows['Data'].map(lambda x: datetime.strptime(str(x), "%d/%m/%Y").strftime('%Y-%m-%d'), na_action='ignore')
dfRows['Dt. Captura'] = dfRows['Dt. Captura'].map(lambda x: datetime.strptime(str(x), "%d/%m/%Y").strftime('%Y-%m-%d'),na_action='ignore')
dfRows['Dt. Situação'] = dfRows['Dt. Situação'].map(lambda x: datetime.strptime(str(x), "%d/%m/%Y").strftime('%Y-%m-%d'),na_action='ignore')
#dfRows['Data'] = dfRows['Data'].map(lambda x:str(x).replace(" 00:00:00", ""))
#dfRows['Data'] = dfRows['Data'].map(lambda x:datetime.strptime(str(x), "%Y-%d-%m").strftime('%Y-%m-%d'), na_action='ignore')



In [9]:
dfRows['Dt. Situação'].unique()

array(['2017-06-01', '2017-06-08', '2017-06-22', '2017-06-02',
       '2017-06-16', '2017-06-09', '2017-06-19', '2017-08-03',
       '2017-07-31', '2017-07-03', '2017-06-05', '2017-06-20',
       '2017-08-15', '2017-08-16', '2017-06-03', '2017-06-21',
       '2017-06-29', '2017-08-02', '2017-06-12', '2017-07-04',
       '2017-06-04', '2017-06-23', '2017-07-13', '2017-06-13',
       '2017-08-05', '2017-06-15', '2017-07-05', '2017-06-06',
       '2017-08-04', '2017-07-06', '2017-06-07', '2017-06-26',
       '2017-07-07', '2017-07-10', '2017-08-07', '2017-05-29',
       '2017-06-27', '2017-08-08', '2017-06-10', '2017-06-28',
       '2017-08-09', '2017-07-11'], dtype=object)

In [11]:
cursor.executemany(
    "INSERT INTO [dbo].[dsTransacaoAdquirente] ([NroAutorizacao] ,[IDERP] ,[IDEQUALS] ,[Adquirente] ,[Tipo] ,[Produto] ,[Estabelecimento] ,[CategoriaEstabelecimento] ,[Bandeira] ,[LoteRO] ,[NroTerminal] ,[MeioCaptura] ,[Data] ,[DtCaptura] ,[Hora] ,[NroCartao] ,[OrigemCartao] ,[NSU] ,[TID] ,[Parc] ,[LoteUnico] ,[NroReferencia] ,[NroFiliacao] ,[VlrBruto] ,[VlrComissao] ,[Taxa] ,[TxContrato] ,[VlrLi­quido] ,[VlrRejeitado] ,[Situacao] ,[DtSituacao] ,[Conciliada] ,[MJustificativa] ,[NomeTarefa] ,[SituacaoTarefa] ,[ConciliacaoParam]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
    dfRows.values.tolist());
conn.commit();